In [ ]:
import main
import lasio
model_path = "..\\tasks\\task 6\\Data\\modelling\\"

#l = lasio.read(model_path + '105.las')
#print(l.curves.keys())
main.create_csv_from_las_modeling(model_path,'modeling_aps.csv')pd.datetime.strptime(x, '%Y.%m.%d')

In [48]:
# данные от организаторов
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'WC'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
water_content_data = water_content_data.rename(columns={'date_start': 'DATE_START_WC'})
water_content_data = water_content_data.rename(columns={'well_name': 'WELL_NAME'})
water_content_data['DATE_START_WC'] = pd.to_datetime(water_content_data['DATE_START_WC'])
water_content_data.head(n=5)

,WELL_NAME,DATE_START_WC,WC
0,380,1905-04-07,0.49
1,379,2006-01-12,0.30
2,370,2007-01-02,0.13
3,417,2007-01-03,0.08
4,389,2007-01-08,0.22


In [71]:
import numpy as np
import pandas as pd
import main
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
perf_data = pd.read_csv(main.data_dir + '\\perforation_wellname.csv' , delimiter=';', date_parser=dateparse)
del perf_data['Пласт']
del perf_data['ГИС']
del perf_data['Тип']
del perf_data['Перфоратор']
del perf_data['N отв']
del perf_data['Hнач(md)']
del perf_data['Hкон(md)']
del perf_data['Hнач(abs)']
del perf_data['Hкон(abs)']
del perf_data['Плотность']
del perf_data['Открыт']
perf_data = perf_data.rename(columns={'Скважина': 'WELL_NAME'})
perf_data = perf_data.rename(columns={'Дата': 'PERF_DATE'})
perf_data = perf_data.rename(columns={'Hнач(abs,тр)':'H_TR_START'})
perf_data = perf_data.rename(columns={'Hкон(abs,тр)':'H_TR_END'})
perf_data['PERF_DATE'] = pd.to_datetime(perf_data['PERF_DATE'])
perf_data.head(n=5)

,WELL_NAME,PERF_DATE,H_TR_START,H_TR_END
0,312,2013-06-28,2510.42,2526.42
1,313,2014-10-06,2523.33,2533.14
2,314,2014-02-04,2507.73,2518.33
3,315ST3,2013-12-29,2525.18,2535.03
4,316,2013-10-16,2522.71,2533.66


In [72]:
merge_data = pd.merge(water_content_data, perf_data, on='WELL_NAME')
merge_data.head()
print(merge_data.count())
merge_data.head(n=2)

WELL_NAME        327
DATE_START_WC    327
WC               327
PERF_DATE        327
H_TR_START       327
H_TR_END         327
dtype: int64


,WELL_NAME,DATE_START_WC,WC,PERF_DATE,H_TR_START,H_TR_END
0,380,1905-04-07,0.49,2012-05-31,2525.77,2538.75
1,379,2006-01-12,0.30,2006-10-22,2512.07,2526.06


In [96]:
merge_data['DIFF'] = (merge_data['DATE_START_WC'] - merge_data['PERF_DATE']).dt.days
print(merge_data.count())
real_data = merge_data[merge_data['DIFF'] > -90]
len(real_data)

WELL_NAME        327
DATE_START_WC    327
WC               327
PERF_DATE        327
H_TR_START       327
H_TR_END         327
DIFF             327
dtype: int64


64

In [98]:
real_data.head(n=2)

,WELL_NAME,DATE_START_WC,WC,PERF_DATE,H_TR_START,H_TR_END,DIFF
3,370,2007-01-02,0.13,2007-02-20,2512.47,2526.46,-49
4,417,2007-01-03,0.08,2007-03-15,2504.91,2517.36,-71
